# P4_Florian_Cazac
## Transportation problem

1. Read an external file containing the parameters of a transportation problem LP problem
2. Different methods can be used to find the initial feasible solution for the problem:
    * Northwest corner rule
    * Minimum cost method
    * Minimum Row cost method
    * Vogel's method
3. the transportation simplex algorithm in an arbitrary transportation problem. 

In [460]:
import pandas as pd
import numpy as np

## 1. Read a file containing the parameters of a transportation problem LP problem

In [461]:
filename = 'transportation_problem.csv'

In [462]:
data = pd.read_csv(filename, index_col=0)
print(data)

supply = data['Supply'][:-1].to_numpy()
demand = data.loc["Demand"].drop("Supply").to_numpy()
cost = data.iloc[:-1, :-1].to_numpy()

print("Supply:", supply)
print("Demand:", demand)
print("Cost:", cost)


         D1   D2   D3   D4  Supply
\                                 
S1       12   13    4    6   500.0
S2        6    4   10   11   700.0
S3       10    9   12    4   800.0
Demand  400  900  200  500     NaN
Supply: [500. 700. 800.]
Demand: [400. 900. 200. 500.]
Cost: [[12 13  4  6]
 [ 6  4 10 11]
 [10  9 12  4]]


## 2. Different methods can be used to find the initial feasible solution

### 1. Northwest corner rule

In [463]:
# North West Corner Method

def north_west_corner(supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    i = 0
    j = 0
    allocation = np.zeros((supply.size, demand.size))
    while i < supply.size and j < demand.size:
        if supply[i] < demand[j]:
            allocation[i][j] = supply[i]
            demand[j] -= supply[i]
            i += 1
        else:
            allocation[i][j] = demand[j]
            supply[i] -= demand[j]
            j += 1
    return allocation

def get_cost(allocation, cost):
    return np.sum(allocation * cost)

In [464]:
north_west_corner_allocation = north_west_corner(supply, demand)
print("NorthWest Corner Method")
print(north_west_corner_allocation)
print("\nCost: ", get_cost(north_west_corner_allocation, cost))

NorthWest Corner Method
[[400. 100.   0.   0.]
 [  0. 700.   0.   0.]
 [  0. 100. 200. 500.]]

Cost:  14200.0


### 2. Minimum cost method

In [465]:
# Minimum Cost Method

def minimum_cost_method(supply, demand, costs):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros_like(costs, dtype=float)  # Allocation matrix
    
    # Get the indices of costs in ascending order
    cost_indices = np.dstack(np.unravel_index(np.argsort(costs, axis=None), costs.shape))[0]
    
    for i, j in cost_indices:
        if supply[i] == 0 or demand[j] == 0:  # Skip exhausted rows/columns
            continue
        
        # Allocate as much as possible to the minimum cost cell
        allocated = min(supply[i], demand[j])
        allocation[i, j] = allocated
        supply[i] -= allocated
        demand[j] -= allocated
    
    return allocation


In [466]:
minimum_cost_allocation = minimum_cost_method(supply, demand, cost)
print("\nMinimum Cost Method")
print(minimum_cost_allocation)
print("\nCost: ", get_cost(minimum_cost_allocation, cost))


Minimum Cost Method
[[300.   0. 200.   0.]
 [  0. 700.   0.   0.]
 [100. 200.   0. 500.]]

Cost:  12000.0


### 3. Minimum Row cost method

In [467]:
def minimum_row_cost_method(supply, demand, cost_matrix):
    """
    Solves the transportation problem using the Minimum Row Cost Method.
    
    Args:
    cost_matrix (list of lists): The cost matrix (rows: suppliers, columns: destinations).
    supply (list): Available supply at each source.
    demand (list): Required demand at each destination.
    
    Returns:
    allocation (2D list): Initial feasible allocation matrix.
    """
    cost_matrix = np.array(cost_matrix, dtype=float)
    supply = supply.copy()  # Avoid modifying the original list
    demand = demand.copy()  # Avoid modifying the original list
    rows, cols = cost_matrix.shape
    allocation = np.zeros((rows, cols), dtype=float)
    
    for i in range(rows):
        while supply[i] > 0:  # While there is supply at row i
            # Find the column with the minimum cost in the current row
            min_cost_index = np.argmin(cost_matrix[i, :])
            
            # Allocate as much as possible to the selected cell
            allocation_quantity = min(supply[i], demand[min_cost_index])
            allocation[i, min_cost_index] = allocation_quantity
            
            # Update supply and demand
            supply[i] -= allocation_quantity
            demand[min_cost_index] -= allocation_quantity
            
            # If demand is met, set the cost to infinity for that column
            if demand[min_cost_index] == 0:
                cost_matrix[:, min_cost_index] = np.inf

    return allocation

In [468]:
minimum_row_cost_allocation = minimum_row_cost_method(supply, demand, cost)
print("\nMinimum Row Cost Method")
print(minimum_row_cost_allocation)
print("\nCost: ", get_cost(minimum_row_cost_allocation, cost))


Minimum Row Cost Method
[[  0.   0. 200. 300.]
 [  0. 700.   0.   0.]
 [400. 200.   0. 200.]]

Cost:  12000.0


### 4. Vogel's method

In [469]:
def calculate_row_penalties(costs, supply):
    """Calculate penalties for each row."""
    supply = supply.copy()
    row_penalties = np.zeros(costs.shape[0])
    for i in range(costs.shape[0]):
        if supply[i] > 0:
            row = costs[i, :]
            sorted_row = np.sort(row)
            if len(sorted_row) > 1:
                row_penalties[i] = sorted_row[1] - sorted_row[0]
            else:
                row_penalties[i] = float('inf')
    return row_penalties

def calculate_column_penalties(costs, demand):
    demand = demand.copy()
    """Calculate penalties for each column."""
    col_penalties = np.zeros(costs.shape[1])
    for j in range(costs.shape[1]):
        if demand[j] > 0:
            col = costs[:, j]
            sorted_col = np.sort(col)
            if len(sorted_col) > 1:
                col_penalties[j] = sorted_col[1] - sorted_col[0]
            else:
                col_penalties[j] = float('inf')
    return col_penalties

In [470]:
def vogels_approximation_method(supply, demand, costs):
    # Convert the costs matrix to float to handle np.inf properly
    supply = supply.copy()
    demand = demand.copy()
    costs = costs.astype(float)
    
    # Initialize the allocation matrix with zeros
    allocation = np.zeros_like(costs, dtype=float)
    
    while np.any(supply > 0) and np.any(demand > 0):
        # Step 1: Calculate penalties for rows and columns
        row_penalties = calculate_row_penalties(costs, supply)
        col_penalties = calculate_column_penalties(costs, demand)
        
        # Step 2: Find the highest penalty and the corresponding row/column
        if np.max(row_penalties) >= np.max(col_penalties):
            # Find the row with the maximum penalty
            row_index = np.argmax(row_penalties)
            col_index = np.argmin(costs[row_index, :])  # Find the column with the minimum cost
        else:
            # Find the column with the maximum penalty
            col_index = np.argmax(col_penalties)
            row_index = np.argmin(costs[:, col_index])  # Find the row with the minimum cost
        
        # Step 3: Allocate as much as possible to the cell
        allocation_amount = min(supply[row_index], demand[col_index])
        allocation[row_index, col_index] = allocation_amount
        supply[row_index] -= allocation_amount
        demand[col_index] -= allocation_amount
        
        # Step 4: If demand or supply is exhausted, set corresponding row/column to np.inf
        if supply[row_index] == 0:
            costs[row_index, :] = np.inf  # Set row to inf
        if demand[col_index] == 0:
            costs[:, col_index] = np.inf  # Set column to inf
    
    return allocation

In [471]:
row_penalities = calculate_row_penalties(cost, supply)
col_penalities = calculate_column_penalties(cost, demand)
print("\nRow Penalties")
print(row_penalities)
print("\nColumn Penalties")
print(col_penalities)

vogels_allocation = vogels_approximation_method(supply, demand, cost)
print("\nVogel's Approximation Method")
print(vogels_allocation)

print("\nCost: ", get_cost(vogels_allocation, cost))


Row Penalties
[2. 2. 5.]

Column Penalties
[4. 5. 6. 2.]

Vogel's Approximation Method
[[  0.   0. 200. 300.]
 [  0. 700.   0.   0.]
 [400. 200.   0. 200.]]

Cost:  12000.0


## 3. Transportation Simplex

In [473]:
def get_us_and_vs(bfs, costs):
    us = np.full(len(costs), None)  # Initialize us as a numpy array with None values
    vs = np.full(len(costs[0]), None)  # Initialize vs as a numpy array with None values
    us[0] = 0  # Assign u[0] = 0 as per the MODI method
    
    bfs_copy = bfs.copy()  # Create a copy of bfs for iteration
    rows, cols = bfs.shape  # Get the shape of the matrix (number of rows and columns)
    
    # Iterate over the bfs matrix to calculate us and vs
    while np.any(bfs_copy != 0):  # Continue while there are still allocations (non-zero values)
        for i in range(rows):
            for j in range(cols):
                if bfs_copy[i, j] != 0:  # If there's an allocation (non-zero value)
                    cost = costs[i, j]
                    if us[i] is None and vs[j] is None:
                        continue  # Skip if both us[i] and vs[j] are None
                    elif us[i] is None:  # If us[i] is None, calculate it using vs[j]
                        us[i] = cost - vs[j]
                    elif vs[j] is None:  # If vs[j] is None, calculate it using us[i]
                        vs[j] = cost - us[i]
                    bfs_copy[i, j] = 0  # Set this allocation to zero (processed)
                    break  # Exit the loop after processing an allocation
        # Repeat the process until no more allocations are left

    return us, vs

In [474]:
def get_ws(bfs, costs, us, vs):
    ws = []
    bfs_indices = np.argwhere(bfs != 0)  # Get indices where there is an allocation (non-zero)
    bfs_set = set(map(tuple, bfs_indices))  # Convert list of indices to a set for quick look-up
    
    rows, cols = costs.shape  # Get the shape of the cost matrix
    
    for i in range(rows):
        for j in range(cols):
            if (i, j) not in bfs_set:  # If the cell (i, j) is not part of the basic feasible solution
                ws_value = us[i] + vs[j] - costs[i, j]
                ws.append(((i, j), ws_value))  # Append the index and the calculated ws value
                
    return ws